In [342]:
# Load the needed libraries
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib notebook
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

import optuna
from hyperopt import tpe, hp, fmin, Trials
import itertools


from scipy.special import inv_boxcox
import warnings
warnings.filterwarnings("ignore")

In [343]:
# read the first 5 rows in the related dataset.
df = pd.read_csv("municipality_bus_utilization.csv")
df.head()

,timestamp,municipality_id,usage,total_capacity
0,2017-06-04 07:59:42,9,454,1332
1,2017-06-04 07:59:42,8,556,2947
2,2017-06-04 07:59:42,4,1090,3893
3,2017-06-04 07:59:42,0,204,2813
4,2017-06-04 07:59:42,7,718,2019


In [344]:
# look at the columns Non-Null Count and their datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13070 entries, 0 to 13069
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   timestamp        13070 non-null  object
 1   municipality_id  13070 non-null  int64 
 2   usage            13070 non-null  int64 
 3   total_capacity   13070 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 408.6+ KB


In [345]:
# convert the timestamp column to datetime. Because it was object datatype.
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [346]:
# extract hour from the timestamp column to create an time_hour column
hour = df['hour'] = df['timestamp'].dt.hour
df['hour']

0         7
1         7
2         7
3         7
4         7
         ..
13065    16
13066    16
13067    16
13068    16
13069    16
Name: hour, Length: 13070, dtype: int64

In [347]:
#extract time/date column from the timestamp column to create an time/date column
time = df['date'] = df['timestamp'].dt.date
time

0        2017-06-04
1        2017-06-04
2        2017-06-04
3        2017-06-04
4        2017-06-04
            ...    
13065    2017-08-19
13066    2017-08-19
13067    2017-08-19
13068    2017-08-19
13069    2017-08-19
Name: timestamp, Length: 13070, dtype: object

In [348]:
# group by the "municipality_id","date", "hour" columns acording to the max value of "usage" column
df1 = df.groupby(["municipality_id","date", "hour"])[["usage"]].max().reset_index()
df1.head()

,municipality_id,date,hour,usage
0,0,2017-06-04,7,204
1,0,2017-06-04,8,332
2,0,2017-06-04,9,485
3,0,2017-06-04,10,583
4,0,2017-06-04,11,614


In [349]:
# Total capacity per municipality
sns.set_style("whitegrid")
plt.figure(figsize = (8, 4))
sns.barplot(x = df["municipality_id"], y = df["total_capacity"])
plt.xlabel('Municipality')
plt.ylabel('Total capacity')
plt.title('Total capacity per municipality')
plt.show()

<IPython.core.display.Javascript object>

In [350]:
# see the each municipality's percent capacity according to the total capacity, and see the total capacity
print("-"* 50)
capacities = df[["municipality_id", "total_capacity"]].drop_duplicates().sort_values("municipality_id")
for i in capacities.iterrows():
    print("Total capacity of the municipality {} = {} ~ {}%".format(
        i[1]["municipality_id"], i[1]["total_capacity"], round((i[1]["total_capacity"]*100)/sum(capacities["total_capacity"]), 2)))
   # print(i, type(i))
print("-"*50)
print("total capacity:", sum(capacities["total_capacity"]))

--------------------------------------------------
Total capacity of the municipality 0 = 2813 ~ 14.26%
Total capacity of the municipality 1 = 397 ~ 2.01%
Total capacity of the municipality 2 = 697 ~ 3.53%
Total capacity of the municipality 3 = 1930 ~ 9.78%
Total capacity of the municipality 4 = 3893 ~ 19.73%
Total capacity of the municipality 5 = 587 ~ 2.98%
Total capacity of the municipality 6 = 3113 ~ 15.78%
Total capacity of the municipality 7 = 2019 ~ 10.23%
Total capacity of the municipality 8 = 2947 ~ 14.94%
Total capacity of the municipality 9 = 1332 ~ 6.75%
--------------------------------------------------
total capacity: 19728


In [351]:
# Distribution of Usages for each municipality
sns.set(style = 'whitegrid')
sns.FacetGrid(df, hue = 'municipality_id', height=6).map(sns.distplot, 'usage').add_legend()
plt.title('Distribution of Usages')
plt.show()

<IPython.core.display.Javascript object>

In [352]:
# take just municipaly_id column and convert to dataframe
municipaly_id = df.iloc[:,1:2].values

municipaly_id = pd.DataFrame(data=municipaly_id, columns = ['municipaly_id'])

print(municipaly_id)

       municipaly_id
0                  9
1                  8
2                  4
3                  0
4                  7
...              ...
13065              2
13066              8
13067              7
13068              6
13069              3

[13070 rows x 1 columns]


In [353]:
# take the 'usage', 'total_capacity' columns values(2nd and 3rd columns)
others = df.iloc[:,2:4].values


In [354]:
#transform the theese columns values and convert to the dataframe
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
impdata = others[:,:]
imputer = imputer.fit(impdata[:,:])
impdata[:,:] = imputer.transform(impdata[:,:])
finalothers = impdata[:,:]
finalothers = pd.DataFrame(data=finalothers, columns = ['usage', 'total_capacity'])
print(finalothers)

       usage  total_capacity
0        454            1332
1        556            2947
2       1090            3893
3        204            2813
4        718            2019
...      ...             ...
13065    548             697
13066   1193            2947
13067   1354            2019
13068   1680            3113
13069   1189            1930

[13070 rows x 2 columns]


In [355]:
#concat the timestamp and municipaly_id columns
data = pd.concat([df['timestamp'], municipaly_id], axis=1)

print(data)

data = pd.concat([data, finalothers], axis=1)

print(data)

                timestamp  municipaly_id
0     2017-06-04 07:59:42              9
1     2017-06-04 07:59:42              8
2     2017-06-04 07:59:42              4
3     2017-06-04 07:59:42              0
4     2017-06-04 07:59:42              7
...                   ...            ...
13065 2017-08-19 16:30:35              2
13066 2017-08-19 16:30:35              8
13067 2017-08-19 16:30:35              7
13068 2017-08-19 16:30:35              6
13069 2017-08-19 16:30:35              3

[13070 rows x 2 columns]
                timestamp  municipaly_id  usage  total_capacity
0     2017-06-04 07:59:42              9    454            1332
1     2017-06-04 07:59:42              8    556            2947
2     2017-06-04 07:59:42              4   1090            3893
3     2017-06-04 07:59:42              0    204            2813
4     2017-06-04 07:59:42              7    718            2019
...                   ...            ...    ...             ...
13065 2017-08-19 16:30:35         

In [356]:
# split the train data until 10390th row and test data after 10390th row and sort values as ascending
traindata = data.iloc[:10390,:]
testdata = data.iloc[10390:,:]

traindata = traindata.sort_values(["municipaly_id","timestamp"], ascending = (True,True))
testdata = testdata.sort_values(["municipaly_id","timestamp"], ascending = (True,True))

print(traindata)

                timestamp  municipaly_id  usage  total_capacity
3     2017-06-04 07:59:42              0    204            2813
18    2017-06-04 08:25:42              0    247            2813
29    2017-06-04 08:59:42              0    332            2813
34    2017-06-04 09:32:46              0    429            2813
47    2017-06-04 09:59:48              0    485            2813
...                   ...            ...    ...             ...
10346 2017-08-02 14:25:50              9    803            1332
10359 2017-08-02 14:58:50              9    763            1332
10364 2017-08-02 15:31:52              9    730            1332
10373 2017-08-02 15:58:52              9    671            1332
10381 2017-08-02 16:32:54              9    625            1332

[10390 rows x 4 columns]


In [357]:
# split the train data according to their municipality_id between the rows
traindata0 = traindata.iloc[:1039,:]
traindata1 = traindata.iloc[1039:2078,:]
traindata2 = traindata.iloc[2078:3117,:]
traindata3 = traindata.iloc[3117:4156,:]
traindata4 = traindata.iloc[4156:5195,:]
traindata5 = traindata.iloc[5195:6234,:]
traindata6 = traindata.iloc[6234:7273,:]
traindata7 = traindata.iloc[7273:8312,:]
traindata8 = traindata.iloc[8312:9351,:]
traindata9 = traindata.iloc[9351:,:]

In [358]:
# print the train datas according to their municipality_id
print(traindata0)

print(traindata1)

print(traindata2)

print(traindata3)

print(traindata4)

print(traindata5)

print(traindata6)

print(traindata7)

print(traindata8)

print(traindata9)

                timestamp  municipaly_id  usage  total_capacity
3     2017-06-04 07:59:42              0    204            2813
18    2017-06-04 08:25:42              0    247            2813
29    2017-06-04 08:59:42              0    332            2813
34    2017-06-04 09:32:46              0    429            2813
47    2017-06-04 09:59:48              0    485            2813
...                   ...            ...    ...             ...
10342 2017-08-02 14:25:50              0    656            2813
10355 2017-08-02 14:58:50              0    646            2813
10361 2017-08-02 15:31:52              0    635            2813
10370 2017-08-02 15:58:52              0    616            2813
10385 2017-08-02 16:32:54              0    638            2813

[1039 rows x 4 columns]
                timestamp  municipaly_id  usage  total_capacity
7     2017-06-04 07:59:42              1    129             397
19    2017-06-04 08:25:42              1    138             397
24    2017-06-0

In [359]:
# split the test datas according to their municipality_id
print(testdata)

testdata0 = testdata.iloc[:268,:]
testdata1 = testdata.iloc[268:536,:]
testdata2 = testdata.iloc[536:804,:]
testdata3 = testdata.iloc[804:1072,:]
testdata4 = testdata.iloc[1072:1340,:]
testdata5 = testdata.iloc[1340:1608,:]
testdata6 = testdata.iloc[1608:1876,:]
testdata7 = testdata.iloc[1876:2144,:]
testdata8 = testdata.iloc[2144:2412,:]
testdata9 = testdata.iloc[2412:,:]


                timestamp  municipaly_id  usage  total_capacity
10398 2017-08-05 08:02:03              0    209            2813
10407 2017-08-05 08:29:08              0    252            2813
10416 2017-08-05 09:02:08              0    322            2813
10425 2017-08-05 09:29:08              0    393            2813
10434 2017-08-05 10:03:04              0    460            2813
...                   ...            ...    ...             ...
13021 2017-08-19 14:30:33              9    983            1332
13035 2017-08-19 15:03:34              9    976            1332
13040 2017-08-19 15:29:33              9    954            1332
13054 2017-08-19 16:03:35              9    879            1332
13063 2017-08-19 16:30:35              9    815            1332

[2680 rows x 4 columns]


In [360]:
# print the test datas according to their municipality_id
print(testdata0)

print(testdata1)

print(testdata2)

print(testdata3)

print(testdata4)

print(testdata5)

print(testdata6)

print(testdata7)

print(testdata8)

print(testdata9)


                timestamp  municipaly_id  usage  total_capacity
10398 2017-08-05 08:02:03              0    209            2813
10407 2017-08-05 08:29:08              0    252            2813
10416 2017-08-05 09:02:08              0    322            2813
10425 2017-08-05 09:29:08              0    393            2813
10434 2017-08-05 10:03:04              0    460            2813
...                   ...            ...    ...             ...
13024 2017-08-19 14:30:33              0   1285            2813
13031 2017-08-19 15:03:34              0   1283            2813
13041 2017-08-19 15:29:33              0   1286            2813
13059 2017-08-19 16:03:35              0   1276            2813
13062 2017-08-19 16:30:35              0   1271            2813

[268 rows x 4 columns]
                timestamp  municipaly_id  usage  total_capacity
10390 2017-08-05 08:02:03              1    141             397
10400 2017-08-05 08:29:08              1    158             397
10410 2017-08-05

According to hours, Selecting Maximum Value and Creating Train and Test Datas

In [361]:
# group by the "municipaly_id" and max values of "usage" columns according to the hour from the "timespent" column for the total dataset
finaldata = traindata.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()
finaldata.head()

,municipaly_id,usage
0,0,204
1,1,129
2,2,273
3,3,623
4,4,1090


In [362]:
# group by the "municipaly_id" and max values of "usage" columns according to the hour from the "timespent" column for the traindata0
finaldata0 = traindata0.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

In [363]:
# group by the "timestamp" and max values of "usage" columns according to the hour from the "timespent" column for the testdata0
tsdata0 = traindata0.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata0)

timestamp
2017-06-04 07:00:00    204.0
2017-06-04 08:00:00    332.0
2017-06-04 09:00:00    485.0
2017-06-04 10:00:00    583.0
2017-06-04 11:00:00    614.0
                       ...  
2017-08-02 12:00:00    661.0
2017-08-02 13:00:00    667.0
2017-08-02 14:00:00    656.0
2017-08-02 15:00:00    635.0
2017-08-02 16:00:00    638.0
Freq: H, Name: usage, Length: 1426, dtype: float64


In [364]:
# group by the "municipaly_id" and max values of "usage" columns according to the hour from the "timespent" column for the finaldata1 variable
finaldata1 = traindata1.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()
finaldata1.head()

,municipaly_id,usage
0,1,129
1,1,164
2,1,245
3,1,317
4,1,341


In [365]:
# group by the "timestamp" and max values of "usage" columns according to the hour from the "timespent" column for the tsdata1 variable
tsdata1 = traindata1.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata1)

timestamp
2017-06-04 07:00:00    129.0
2017-06-04 08:00:00    164.0
2017-06-04 09:00:00    245.0
2017-06-04 10:00:00    317.0
2017-06-04 11:00:00    341.0
                       ...  
2017-08-02 12:00:00    411.0
2017-08-02 13:00:00    411.0
2017-08-02 14:00:00    406.0
2017-08-02 15:00:00    391.0
2017-08-02 16:00:00    358.0
Freq: H, Name: usage, Length: 1426, dtype: float64


In [366]:
# group by the "municipaly_id" and max values of "usage" columns according to the hour from the "timespent" column for the finaldata2 variable
finaldata2 = traindata2.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

In [367]:
# group by the "timestamp" and max values of "usage" columns according to the hour from the "timespent" column for the tsdata1 variable
tsdata2 = traindata2.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata2)

timestamp
2017-06-04 07:00:00    273.0
2017-06-04 08:00:00    355.0
2017-06-04 09:00:00    494.0
2017-06-04 10:00:00    582.0
2017-06-04 11:00:00    643.0
                       ...  
2017-08-02 12:00:00    683.0
2017-08-02 13:00:00    694.0
2017-08-02 14:00:00    653.0
2017-08-02 15:00:00    593.0
2017-08-02 16:00:00    553.0
Freq: H, Name: usage, Length: 1426, dtype: float64


In [368]:
# group by the "municipaly_id" and max values of "usage" columns according to the hour from the "timespent" column for the finaldata3 variable
finaldata3 = traindata3.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

In [369]:
# group by the "timestamp" and max values of "usage" columns according to the hour from the "timespent" column for the tsdata3 variable
tsdata3 = traindata3.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata3)

timestamp
2017-06-04 07:00:00     623.0
2017-06-04 08:00:00     967.0
2017-06-04 09:00:00    1263.0
2017-06-04 10:00:00    1396.0
2017-06-04 11:00:00    1456.0
                        ...  
2017-08-02 12:00:00    1408.0
2017-08-02 13:00:00    1378.0
2017-08-02 14:00:00    1342.0
2017-08-02 15:00:00    1185.0
2017-08-02 16:00:00     952.0
Freq: H, Name: usage, Length: 1426, dtype: float64


In [370]:
# group by the "municipaly_id" and max values of "usage" columns according to the hour from the "timespent" column for the finaldata4 variable
finaldata4 = traindata4.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

In [371]:
# group by the "timestamp" and max values of "usage" columns according to the hour from the "timespent" column for the tsdata4 variable
tsdata4 = traindata4.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata4)

timestamp
2017-06-04 07:00:00    1090.0
2017-06-04 08:00:00    2016.0
2017-06-04 09:00:00    2811.0
2017-06-04 10:00:00    3178.0
2017-06-04 11:00:00    3257.0
                        ...  
2017-08-02 12:00:00    3052.0
2017-08-02 13:00:00    3018.0
2017-08-02 14:00:00    2959.0
2017-08-02 15:00:00    2641.0
2017-08-02 16:00:00    2152.0
Freq: H, Name: usage, Length: 1426, dtype: float64


In [372]:
# group by the "municipaly_id" and max values of "usage" columns according to the hour from the "timespent" column for the finaldata5 variable
finaldata5 = traindata5.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

In [373]:
# group by the "timestamp" and max values of "usage" columns according to the hour from the "timespent" column for the tsdata5 variable
tsdata5 = traindata5.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata5)

timestamp
2017-06-04 07:00:00     70.0
2017-06-04 08:00:00     89.0
2017-06-04 09:00:00    159.0
2017-06-04 10:00:00    228.0
2017-06-04 11:00:00    268.0
                       ...  
2017-08-02 12:00:00    209.0
2017-08-02 13:00:00    212.0
2017-08-02 14:00:00    195.0
2017-08-02 15:00:00    158.0
2017-08-02 16:00:00    114.0
Freq: H, Name: usage, Length: 1426, dtype: float64


In [374]:
# group by the "municipaly_id" and max values of "usage" columns according to the hour from the "timespent" column for the finaldata6 variable
finaldata6 = traindata6.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

In [375]:
# group by the "timestamp" and max values of "usage" columns according to the hour from the "timespent" column for the tsdata6 variable
tsdata6 = traindata6.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata6)

timestamp
2017-06-04 07:00:00     597.0
2017-06-04 08:00:00     801.0
2017-06-04 09:00:00    1061.0
2017-06-04 10:00:00    1219.0
2017-06-04 11:00:00    1297.0
                        ...  
2017-08-02 12:00:00    1167.0
2017-08-02 13:00:00    1182.0
2017-08-02 14:00:00    1144.0
2017-08-02 15:00:00    1059.0
2017-08-02 16:00:00    1018.0
Freq: H, Name: usage, Length: 1426, dtype: float64


In [376]:
# group by the "municipaly_id" and max values of "usage" columns according to the hour from the "timespent" column for the finaldata7 variable
finaldata7 = traindata7.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

In [377]:
# group by the "timestamp" and max values of "usage" columns according to the hour from the "timespent" column for the tsdata7 variable
tsdata7 = traindata7.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata7)

timestamp
2017-06-04 07:00:00     718.0
2017-06-04 08:00:00     873.0
2017-06-04 09:00:00    1151.0
2017-06-04 10:00:00    1373.0
2017-06-04 11:00:00    1479.0
                        ...  
2017-08-02 12:00:00    1493.0
2017-08-02 13:00:00    1509.0
2017-08-02 14:00:00    1447.0
2017-08-02 15:00:00    1314.0
2017-08-02 16:00:00    1169.0
Freq: H, Name: usage, Length: 1426, dtype: float64


In [378]:
# group by the "municipaly_id" and max values of "usage" columns according to the hour from the "timespent" column for the finaldata8 variable
finaldata8 = traindata8.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

In [379]:
# group by the "timestamp" and max values of "usage" columns according to the hour from the "timespent" column for the tsdata8 variable
tsdata8 = traindata8.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tsdata8)

timestamp
2017-06-04 07:00:00     556.0
2017-06-04 08:00:00     857.0
2017-06-04 09:00:00    1172.0
2017-06-04 10:00:00    1363.0
2017-06-04 11:00:00    1399.0
                        ...  
2017-08-02 12:00:00    1268.0
2017-08-02 13:00:00    1288.0
2017-08-02 14:00:00    1247.0
2017-08-02 15:00:00    1172.0
2017-08-02 16:00:00    1035.0
Freq: H, Name: usage, Length: 1426, dtype: float64


In [380]:
# group by the "municipaly_id" and max values of "usage" columns according to the hour from the "timespent" column for the finaldata9 variable
finaldata9 = traindata9.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

In [381]:
# group by the "timestamp" and max values of "usage" columns according to the hour from the "timespent" column for the tsdata9 variable
tsdata9 = traindata9.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

In [382]:
#pribnt the tsdata9 and finaldatas from 0 to 9 variables
print(tsdata9)

print(finaldata)

print(finaldata0)

print(finaldata1)

print(finaldata2)

print(finaldata3)

print(finaldata4)

print(finaldata5)

print(finaldata6)

print(finaldata7)

print(finaldata8)

print(finaldata9)


timestamp
2017-06-04 07:00:00    454.0
2017-06-04 08:00:00    527.0
2017-06-04 09:00:00    666.0
2017-06-04 10:00:00    800.0
2017-06-04 11:00:00    845.0
                       ...  
2017-08-02 12:00:00    819.0
2017-08-02 13:00:00    824.0
2017-08-02 14:00:00    803.0
2017-08-02 15:00:00    730.0
2017-08-02 16:00:00    625.0
Freq: H, Name: usage, Length: 1426, dtype: float64
      municipaly_id  usage
0                 0    204
1                 1    129
2                 2    273
3                 3    623
4                 4   1090
...             ...    ...
5315              5    114
5316              6   1018
5317              7   1169
5318              8   1035
5319              9    625

[5320 rows x 2 columns]
     municipaly_id  usage
0                0    204
1                0    332
2                0    485
3                0    583
4                0    614
..             ...    ...
527              0    661
528              0    667
529              0    656
530        

In [383]:
# group by the "timestamp", "municipaly_id" and max values of "usage" columns according to the hour from the "timespent" column for the trydata and trydata0 variable
trydata = testdata.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

trydata0 = testdata0.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata0 = testdata0.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()
print(trydata0)

     municipaly_id  usage
0                0    252
1                0    393
2                0    517
3                0    653
4                0    655
..             ...    ...
137              0   1281
138              0   1283
139              0   1285
140              0   1286
141              0   1276

[142 rows x 2 columns]


In [384]:
trydata1 = testdata1.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata1 = testdata1.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tstdata1)

timestamp
2017-08-05 08:00:00    158.0
2017-08-05 09:00:00    216.0
2017-08-05 10:00:00    300.0
2017-08-05 11:00:00    397.0
2017-08-05 12:00:00    398.0
                       ...  
2017-08-19 12:00:00    406.0
2017-08-19 13:00:00    408.0
2017-08-19 14:00:00    410.0
2017-08-19 15:00:00    411.0
2017-08-19 16:00:00    401.0
Freq: H, Name: usage, Length: 345, dtype: float64


In [385]:
trydata2 = testdata2.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata2 = testdata2.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tstdata2)

timestamp
2017-08-05 08:00:00    327.0
2017-08-05 09:00:00    405.0
2017-08-05 10:00:00    483.0
2017-08-05 11:00:00    567.0
2017-08-05 12:00:00    584.0
                       ...  
2017-08-19 12:00:00    663.0
2017-08-19 13:00:00    697.0
2017-08-19 14:00:00    687.0
2017-08-19 15:00:00    651.0
2017-08-19 16:00:00    580.0
Freq: H, Name: usage, Length: 345, dtype: float64


In [386]:
trydata3 = testdata3.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata3 = testdata3.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tstdata3)

timestamp
2017-08-05 08:00:00     781.0
2017-08-05 09:00:00    1121.0
2017-08-05 10:00:00    1289.0
2017-08-05 11:00:00    1557.0
2017-08-05 12:00:00    1569.0
                        ...  
2017-08-19 12:00:00    1484.0
2017-08-19 13:00:00    1519.0
2017-08-19 14:00:00    1530.0
2017-08-19 15:00:00    1496.0
2017-08-19 16:00:00    1330.0
Freq: H, Name: usage, Length: 345, dtype: float64


In [387]:
trydata4 = testdata4.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata4 = testdata4.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tstdata4)

timestamp
2017-08-05 08:00:00    2132.0
2017-08-05 09:00:00    2814.0
2017-08-05 10:00:00    3123.0
2017-08-05 11:00:00    3447.0
2017-08-05 12:00:00    3461.0
                        ...  
2017-08-19 12:00:00    3157.0
2017-08-19 13:00:00    3194.0
2017-08-19 14:00:00    3183.0
2017-08-19 15:00:00    3111.0
2017-08-19 16:00:00    2779.0
Freq: H, Name: usage, Length: 345, dtype: float64


In [388]:
trydata5 = testdata5.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata5 = testdata5.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tstdata5)

timestamp
2017-08-05 08:00:00     23.0
2017-08-05 09:00:00     48.0
2017-08-05 10:00:00     96.0
2017-08-05 11:00:00    157.0
2017-08-05 12:00:00    170.0
                       ...  
2017-08-19 12:00:00    258.0
2017-08-19 13:00:00    300.0
2017-08-19 14:00:00    318.0
2017-08-19 15:00:00    309.0
2017-08-19 16:00:00    239.0
Freq: H, Name: usage, Length: 345, dtype: float64


In [389]:
trydata6 = testdata6.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata6 = testdata6.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tstdata6)

timestamp
2017-08-05 08:00:00     571.0
2017-08-05 09:00:00     755.0
2017-08-05 10:00:00     903.0
2017-08-05 11:00:00    1057.0
2017-08-05 12:00:00    1082.0
                        ...  
2017-08-19 12:00:00    1787.0
2017-08-19 13:00:00    1845.0
2017-08-19 14:00:00    1853.0
2017-08-19 15:00:00    1829.0
2017-08-19 16:00:00    1727.0
Freq: H, Name: usage, Length: 345, dtype: float64


In [390]:
trydata7 = testdata7.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata7 = testdata7.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tstdata7)

timestamp
2017-08-05 08:00:00     651.0
2017-08-05 09:00:00     820.0
2017-08-05 10:00:00    1038.0
2017-08-05 11:00:00    1319.0
2017-08-05 12:00:00    1374.0
                        ...  
2017-08-19 12:00:00    1503.0
2017-08-19 13:00:00    1627.0
2017-08-19 14:00:00    1657.0
2017-08-19 15:00:00    1618.0
2017-08-19 16:00:00    1450.0
Freq: H, Name: usage, Length: 345, dtype: float64


In [391]:
trydata8 = testdata8.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata8 = testdata8.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

print(tstdata8)

timestamp
2017-08-05 08:00:00     572.0
2017-08-05 09:00:00     847.0
2017-08-05 10:00:00    1059.0
2017-08-05 11:00:00    1249.0
2017-08-05 12:00:00    1264.0
                        ...  
2017-08-19 12:00:00    1251.0
2017-08-19 13:00:00    1315.0
2017-08-19 14:00:00    1341.0
2017-08-19 15:00:00    1336.0
2017-08-19 16:00:00    1253.0
Freq: H, Name: usage, Length: 345, dtype: float64


In [392]:
trydata9 = testdata9.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipaly_id'], as_index=False)['usage'].max()

tstdata9 = testdata9.groupby([pd.Grouper(key='timestamp', freq='H')])['usage'].max()

In [393]:
print(tstdata9)

print(trydata)

print(trydata0)

print(trydata1)

print(trydata2)

print(trydata3)

print(trydata4)

print(trydata5)

print(trydata6)

print(trydata7)

print(trydata8)

print(trydata9)


timestamp
2017-08-05 08:00:00    333.0
2017-08-05 09:00:00    424.0
2017-08-05 10:00:00    564.0
2017-08-05 11:00:00    761.0
2017-08-05 12:00:00    799.0
                       ...  
2017-08-19 12:00:00    849.0
2017-08-19 13:00:00    941.0
2017-08-19 14:00:00    983.0
2017-08-19 15:00:00    976.0
2017-08-19 16:00:00    879.0
Freq: H, Name: usage, Length: 345, dtype: float64
      municipaly_id  usage
0                 0    252
1                 1    158
2                 2    327
3                 3    781
4                 4   2132
...             ...    ...
1415              5    239
1416              6   1727
1417              7   1450
1418              8   1253
1419              9    879

[1420 rows x 2 columns]
     municipaly_id  usage
0                0    252
1                0    393
2                0    517
3                0    653
4                0    655
..             ...    ...
137              0   1281
138              0   1283
139              0   1285
140         

Forecasting and Plotting

In [394]:
warnings.filterwarnings("ignore")

# Predict0
model0 = ExponentialSmoothing(finaldata0["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model0 = model0.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred0 = hw_model0.predict(start=trydata0["usage"].index[0], end=trydata0["usage"].index[-1])

pred0.index += len(finaldata0)

trydata0.index += len(finaldata0)

print(pred0)



532    358.600000
533    204.773000
534    331.363865
535    484.231819
536    582.506159
          ...    
669    696.949041
670    691.029745
671    679.060149
672    630.245301
673    548.411227
Length: 142, dtype: float64


In [395]:
plt.figure(figsize=(20, 6), dpi=200);plt.plot(finaldata0["usage"].index, finaldata0["usage"], label='Train'); plt.plot(pred0.index, pred0, 'b-', label = 'Predicted Data'); plt.plot(trydata0["usage"].index, trydata0["usage"], 'r-', label = 'Real Data'); plt.xlabel('Hours'); plt.ylabel('Bus'); plt.title('Forecasting 0'); plt.legend(loc="best");

<IPython.core.display.Javascript object>

In [396]:
# Predict1
model1 = ExponentialSmoothing(finaldata1["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model1 = model1.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred1 = hw_model1.predict(start=trydata1["usage"].index[0], end=trydata1["usage"].index[-1])

pred1.index += len(finaldata1)

trydata1.index += len(finaldata1)

print(pred1)

532    129.007087
533    129.000000
534    163.999999
535    244.999999
536    316.999999
          ...    
669    349.000000
670    341.000000
671    327.000000
672    266.000001
673    215.000001
Length: 142, dtype: float64


In [397]:
plt.figure(figsize=(20, 6), dpi=200);plt.plot(finaldata1["usage"].index, finaldata1["usage"], label='Train'); plt.plot(pred1.index, pred1, 'b-', label = 'Predicted Data'); plt.plot(trydata1["usage"].index, trydata1["usage"], 'r-', label = 'Real Data'); plt.xlabel('Hours'); plt.ylabel('Bus'); plt.title('Forecasting 1'); plt.legend(loc="best");

<IPython.core.display.Javascript object>

In [398]:
# Predict2
model2 = ExponentialSmoothing(finaldata2["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model2 = model2.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred2 = hw_model2.predict(start=trydata2["usage"].index[0], end=trydata2["usage"].index[-1])

pred2.index += len(finaldata2)

trydata2.index += len(finaldata2)

print(pred2)

532    336.853481
533    273.000001
534    354.999999
535    493.999998
536    581.999999
          ...    
669    625.000000
670    594.000000
671    563.000000
672    477.000001
673    399.000001
Length: 142, dtype: float64


In [399]:
plt.figure(figsize=(20, 6), dpi=200);plt.plot(finaldata2["usage"].index, finaldata2["usage"], label='Train'); plt.plot(pred2.index, pred2, 'b-', label = 'Predicted Data'); plt.plot(trydata2["usage"].index, trydata2["usage"], 'r-', label = 'Real Data'); plt.xlabel('Hours'); plt.ylabel('Bus'); plt.title('Predicted Value 2'); plt.legend(loc="best");

<IPython.core.display.Javascript object>

In [400]:
# Predict3
model3 = ExponentialSmoothing(finaldata3["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model3 = model3.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred3 = hw_model3.predict(start=trydata3["usage"].index[0], end=trydata3["usage"].index[-1])

pred3.index += len(finaldata3)

trydata3.index += len(finaldata3)

print(pred3)

532     963.733333
533     624.703667
534     965.288518
535    1261.511443
536    1395.327557
          ...     
669    1417.999018
670    1381.184995
671    1338.215925
672    1173.826080
673     966.044130
Length: 142, dtype: float64


In [401]:
plt.figure(figsize=(20, 6), dpi=200);plt.plot(finaldata3["usage"].index, finaldata3["usage"], label='Train'); plt.plot(pred3.index, pred3, 'b-', label = 'Predicted Data'); plt.plot(trydata3["usage"].index, trydata3["usage"], 'r-', label = 'Real Data'); plt.xlabel('Hours'); plt.ylabel('Bus'); plt.title('Forecasting 3'); plt.legend(loc="best");

<IPython.core.display.Javascript object>

In [402]:
# Predict4
model4 = ExponentialSmoothing(finaldata4["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model4 = model4.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred4 = hw_model4.predict(start=trydata4["usage"].index[0], end=trydata4["usage"].index[-1])

pred4.index += len(finaldata4)

trydata4.index += len(finaldata4)

print(pred4)


532    2021.800000
533    1094.659000
534    2011.393295
535    2807.001966
536    3176.145010
          ...     
669    3139.924132
670    3052.439621
671    2957.477198
672    2444.577386
673    2038.042887
Length: 142, dtype: float64


In [403]:
plt.figure(figsize=(20, 6), dpi=200);plt.plot(finaldata4["usage"].index, finaldata4["usage"], label='Train'); plt.plot(pred4.index, pred4, 'b-', label = 'Predicted Data'); plt.plot(trydata4["usage"].index, trydata4["usage"], 'r-', label = 'Real Data'); plt.xlabel('Hours'); plt.ylabel('Bus'); plt.title('Forecasting 4'); plt.legend(loc="best");

<IPython.core.display.Javascript object>

In [404]:
# Predict5
model5 = ExponentialSmoothing(finaldata5["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model5 = model5.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred5 = hw_model5.predict(start=trydata5["usage"].index[0], end=trydata5["usage"].index[-1])

pred5.index += len(finaldata5)

trydata5.index += len(finaldata5)

print(pred5)

532    125.866667
533     70.279333
534     88.906397
535    158.649532
536    227.653248
          ...    
669    245.969919
670    209.184850
671    174.175924
672    133.205880
673     68.326029
Length: 142, dtype: float64


In [405]:
plt.figure(figsize=(20, 6), dpi=200);plt.plot(finaldata5["usage"].index, finaldata5["usage"], label='Train'); plt.plot(pred5.index, pred5, 'b-', label = 'Predicted Data'); plt.plot(trydata5["usage"].index, trydata5["usage"], 'r-', label = 'Real Data'); plt.xlabel('Hours'); plt.ylabel('Bus'); plt.title('Forecasting 5'); plt.legend(loc="best");

<IPython.core.display.Javascript object>

In [406]:
# Predict6
model6 = ExponentialSmoothing(finaldata6["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model6 = model6.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred6 = hw_model6.predict(start=trydata6["usage"].index[0], end=trydata6["usage"].index[-1])

pred6.index += len(finaldata6)

trydata6.index += len(finaldata6)

print(pred6)

532     850.733333
533     598.268667
534     799.986343
535    1059.694932
536    1218.203475
          ...     
669    1350.903764
670    1318.164519
671    1278.200823
672    1163.576004
673    1016.737880
Length: 142, dtype: float64


In [407]:
plt.figure(figsize=(20, 6), dpi=200);plt.plot(finaldata6["usage"].index, finaldata6["usage"], label='Train'); plt.plot(pred6.index, pred6, 'b-', label = 'Predicted Data'); plt.plot(trydata6["usage"].index, trydata6["usage"], 'r-', label = 'Real Data'); plt.xlabel('Hours'); plt.ylabel('Bus'); plt.title('Forecasting 6'); plt.legend(loc="best");

<IPython.core.display.Javascript object>

In [408]:
# Predict7
model7 = ExponentialSmoothing(finaldata7["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model7 = model7.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred7 = hw_model7.predict(start=trydata7["usage"].index[0], end=trydata7["usage"].index[-1])

pred7.index += len(finaldata7)

trydata7.index += len(finaldata7)

print(pred7)

532     949.333333
533     719.156667
534     872.230783
535    1149.606154
536    1371.883031
          ...     
669    1422.914264
670    1367.279571
671    1310.286398
672    1117.966432
673     919.994832
Length: 142, dtype: float64


In [409]:
plt.figure(figsize=(20, 6), dpi=200);plt.plot(finaldata7["usage"].index, finaldata7["usage"], label='Train'); plt.plot(pred7.index, pred7, 'b-', label = 'Predicted Data'); plt.plot(trydata7["usage"].index, trydata7["usage"], 'r-', label = 'Real Data'); plt.xlabel('Hours'); plt.ylabel('Bus'); plt.title('Forecasting 7'); plt.legend(loc="best");

<IPython.core.display.Javascript object>

In [410]:
# Predict8
model8 = ExponentialSmoothing(finaldata8["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model8 = model8.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred8 = hw_model8.predict(start=trydata8["usage"].index[0], end=trydata8["usage"].index[-1])

pred8.index += len(finaldata8)

trydata8.index += len(finaldata8)

print(pred8)

532     893.800000
533     557.689000
534     855.503445
535    1170.417517
536    1362.037088
          ...     
669    1417.984766
670    1372.229924
671    1328.221150
672    1114.076106
673     949.825381
Length: 142, dtype: float64


In [411]:
plt.figure(figsize=(20, 6), dpi=200);plt.plot(finaldata8["usage"].index, finaldata8["usage"], label='Train'); plt.plot(pred8.index, pred8, 'b-', label = 'Predicted Data'); plt.plot(trydata8["usage"].index, trydata8["usage"], 'r-', label = 'Real Data'); plt.xlabel('Hours'); plt.ylabel('Bus'); plt.title('Forecasting 8'); plt.legend(loc="best");

<IPython.core.display.Javascript object>

In [412]:
# Predict9
model9 = ExponentialSmoothing(finaldata9["usage"], trend=None, seasonal=None, seasonal_periods=None)
hw_model9 = model9.fit(optimized=True, use_boxcox=None, remove_bias=False)
pred9 = hw_model9.predict(start=trydata9["usage"].index[0], end=trydata9["usage"].index[-1])

pred9.index += len(finaldata9)

trydata9.index += len(finaldata9)

print(pred9)

532    577.666667
533    454.618333
534    526.638092
535    665.303190
536    799.326516
          ...    
669    814.994620
670    782.164973
671    756.130825
672    649.535654
673    529.602678
Length: 142, dtype: float64


In [413]:
plt.figure(figsize=(20, 6), dpi=200);plt.plot(finaldata9["usage"].index, finaldata9["usage"], label='Train'); plt.plot(pred9.index, pred9, 'b-', label = 'Predicted Data'); plt.plot(trydata9["usage"].index, trydata9["usage"], 'r-', label = 'Real Data'); plt.xlabel('Hours'); plt.ylabel('Bus'); plt.title('Forecasting 9'); plt.legend(loc="best");

<IPython.core.display.Javascript object>

LGBM

In [414]:
df1 = pd.read_csv("municipality_bus_utilization.csv")
df1.head()

,timestamp,municipality_id,usage,total_capacity
0,2017-06-04 07:59:42,9,454,1332
1,2017-06-04 07:59:42,8,556,2947
2,2017-06-04 07:59:42,4,1090,3893
3,2017-06-04 07:59:42,0,204,2813
4,2017-06-04 07:59:42,7,718,2019


In [415]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13070 entries, 0 to 13069
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   timestamp        13070 non-null  object
 1   municipality_id  13070 non-null  int64 
 2   usage            13070 non-null  int64 
 3   total_capacity   13070 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 408.6+ KB


In [416]:
def check_df(df1):
    print(" SHAPE ".center(70, "~"))
    print(df1.shape)
    print(" TYPES ".center(70, "~"))
    print(df1.dtypes)
    print(" HEAD ".center(70, "~"))
    display(df1.head())
    print(" TAIL ".center(70, "~"))
    display(df1.tail())
    print(" INFO ".center(70, "~"))
    print(df1.info())
    print(" NA ".center(70, "~"))
    print(df1.isnull().sum())
    print(" DESCRIBE ".center(70, "~"))
    display(df1.describe().T)
    print(" QUANTILES ".center(70, "~"))
    display(df1.describe([0, 0.05, 0.50, 0.95, 0.99]).T)

check_df(df1)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ SHAPE ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(13070, 4)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ TYPES ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
timestamp          object
municipality_id     int64
usage               int64
total_capacity      int64
dtype: object
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ HEAD ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


,timestamp,municipality_id,usage,total_capacity
0,2017-06-04 07:59:42,9,454,1332
1,2017-06-04 07:59:42,8,556,2947
2,2017-06-04 07:59:42,4,1090,3893
3,2017-06-04 07:59:42,0,204,2813
4,2017-06-04 07:59:42,7,718,2019


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ TAIL ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


,timestamp,municipality_id,usage,total_capacity
13065,2017-08-19 16:30:35,2,548,697
13066,2017-08-19 16:30:35,8,1193,2947
13067,2017-08-19 16:30:35,7,1354,2019
13068,2017-08-19 16:30:35,6,1680,3113
13069,2017-08-19 16:30:35,3,1189,1930


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ INFO ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13070 entries, 0 to 13069
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   timestamp        13070 non-null  object
 1   municipality_id  13070 non-null  int64 
 2   usage            13070 non-null  int64 
 3   total_capacity   13070 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 408.6+ KB
None
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ NA ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
timestamp          0
municipality_id    0
usage              0
total_capacity     0
dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ DESCRIBE ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


,count,mean,std,min,25%,50%,75%,max
municipality_id,13070.0,4.500000,2.872391,0.0,2.0,4.5,7.0,9.0
usage,13070.0,876.002984,679.287109,11.0,403.0,692.0,1190.0,3508.0
total_capacity,13070.0,1972.800000,1144.074782,397.0,697.0,1974.5,2947.0,3893.0


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ QUANTILES ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


,count,mean,std,min,0%,5%,50%,95%,99%,max
municipality_id,13070.0,4.500000,2.872391,0.0,0.0,0.0,4.5,9.00,9.00,9.0
usage,13070.0,876.002984,679.287109,11.0,11.0,141.0,692.0,2558.55,3318.31,3508.0
total_capacity,13070.0,1972.800000,1144.074782,397.0,397.0,397.0,1974.5,3893.00,3893.00,3893.0


In [417]:
df1["timestamp"] = pd.to_datetime(df1["timestamp"])

In [418]:
df1["hour"] = df1['timestamp'].dt.hour
df1["date"] = df1['timestamp'].dt.date

In [419]:
df1 = df1.groupby(["municipality_id","date", "hour"])[["usage"]].max().reset_index()
check_df(df1)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ SHAPE ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(6740, 4)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ TYPES ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
municipality_id     int64
date               object
hour                int64
usage               int64
dtype: object
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ HEAD ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


,municipality_id,date,hour,usage
0,0,2017-06-04,7,204
1,0,2017-06-04,8,332
2,0,2017-06-04,9,485
3,0,2017-06-04,10,583
4,0,2017-06-04,11,614


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ TAIL ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


,municipality_id,date,hour,usage
6735,9,2017-08-19,12,849
6736,9,2017-08-19,13,941
6737,9,2017-08-19,14,983
6738,9,2017-08-19,15,976
6739,9,2017-08-19,16,879


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ INFO ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6740 entries, 0 to 6739
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   municipality_id  6740 non-null   int64 
 1   date             6740 non-null   object
 2   hour             6740 non-null   int64 
 3   usage            6740 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 210.8+ KB
None
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ NA ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
municipality_id    0
date               0
hour               0
usage              0
dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ DESCRIBE ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


,count,mean,std,min,25%,50%,75%,max
municipality_id,6740.0,4.500000,2.872494,0.0,2.00,4.5,7.00,9.0
hour,6740.0,11.807122,2.708833,7.0,9.00,12.0,14.00,16.0
usage,6740.0,893.321662,687.228407,12.0,408.75,709.0,1206.25,3508.0


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ QUANTILES ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


,count,mean,std,min,0%,5%,50%,95%,99%,max
municipality_id,6740.0,4.500000,2.872494,0.0,0.0,0.0,4.5,9.0,9.00,9.0
hour,6740.0,11.807122,2.708833,7.0,7.0,8.0,12.0,16.0,16.00,16.0
usage,6740.0,893.321662,687.228407,12.0,12.0,148.9,709.0,2621.5,3327.83,3508.0


In [420]:
test = df1.loc[df1["date"] >= pd.to_datetime("2017-08-05")]
train = df1.loc[df1["date"] < pd.to_datetime("2017-08-05")]

print(f"Shape of Dataframe: {df1.shape}")
print(f"Shape of Train Set: {train.shape}")
print(f"Shape of Test Set: {test.shape}")

Shape of Dataframe: (6740, 4)
Shape of Train Set: (5320, 4)
Shape of Test Set: (1420, 4)


In [421]:
train_sets = dict()
test_sets = dict()
for i in range(df1.municipality_id.nunique()):
    train_sets[f"train{i}"] = train.loc[train.municipality_id == i]
    test_sets[f"test{i}"] = test.loc[test.municipality_id == i]

In [422]:
rmse_scores_lgb = []
mae_scores_lgb = []
def lgbm_base(train, test):
    global rmse_scores_lgb, mae_scores_lgb
    params = LGBMRegressor().get_params()  # default lgbm params
    del params["silent"]
    del params["importance_type"]
    params["verbosity"] = -99
    lgb_train = lgb.Dataset(train_sets[train].drop("date", axis=1), label=train_sets[train]["usage"])
    lgb_val = lgb.Dataset(test_sets[test].drop("date", axis=1), label=test_sets[test]["usage"])
    model = lgb.train(params, lgb_train,
                      valid_sets=[lgb_train, lgb_val],
                      verbose_eval=-99)
    y_pred = model.predict(test_sets[test].drop("date", axis=1), num_iteration=model.best_iteration)

    print(f"Results for {train[-1]}. Municipality")
    rmse = np.sqrt(mean_squared_error(test_sets[test]['usage'], y_pred))
    rmse_scores_lgb.append(rmse)
    print(f"RMSE: {rmse}")

    mae = mean_absolute_error(test_sets[test]['usage'], y_pred)
    mae_scores_lgb.append(mae)
    print(f"MAE: {mae}")
    print("".center(50, "~"))

In [423]:
for tr, te in zip(train_sets.keys(), test_sets.keys()):
    lgbm_base(tr, te)

print(f"Average RMSE: {np.mean(rmse_scores_lgb)}")
print(f"Average MAE: {np.mean(mae_scores_lgb)}")

Results for 0. Municipality
RMSE: 24.585615634112727
MAE: 11.069223018239244
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Results for 1. Municipality
RMSE: 1.0058173750128196
MAE: 0.6774969588929431
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Results for 2. Municipality
RMSE: 4.001680610730357
MAE: 1.8504827868753615
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Results for 3. Municipality
RMSE: 9.82295502471878
MAE: 5.560097375623665
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Results for 4. Municipality
RMSE: 14.945770265295417
MAE: 9.371807842179955
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Results for 5. Municipality
RMSE: 6.75106367254838
MAE: 3.181790228502604
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Results for 6. Municipality
RMSE: 25.474796733353575
MAE: 9.060595283483858
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Results for 7. Municipality
RMSE: 26.96539816274999
MAE: 13.066867944024372
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Hyperparameter Tunings

In [424]:
def plot_scores(before, after):
    labels = range(10)
    x = np.arange(len(before))  # the label locations
    width = 0.4  # the width of the bars
    fig, ax = plt.subplots()
    ax.bar(x - width/2, before, width, label='Before')
    ax.bar(x + width/2, after, width, label='After')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Scores')
    ax.set_xlabel("Municipality")
    ax.set_title('Before Tuning - After Tuning (Lower is better)')
    ax.set_xticks(labels)
    ax.legend()
    fig.tight_layout()
    plt.show();

LGBM Hyperparameter Tuning

In [425]:
params = dict(
    max_depth = hp.randint("max_depth", 2, 15),
    #early_stopping_rounds = hp.randint("early_stopping_rounds", 2, 350),
    feature_fraction = hp.uniform("feature_fraction", 0.5, 1),
    learning_rate = hp.uniform("learning_rate", 0.0001, 0.2),
    num_boost_round= hp.randint("num_boost_round", 50, 2500),
    num_leaves = hp.randint("num_leaves", 10, 100),
    n_jobs = -1,
    random_state=41,
    verbosity=-9999)

In [426]:



def lgbm_objective(params, train, test):
    lgb_train = lgb.Dataset(train_sets[train].drop("date", axis=1), label=train_sets[train]["usage"])
    lgb_val = lgb.Dataset(test_sets[test].drop("date", axis=1), label=test_sets[test]["usage"])
    model = lgb.train(params, lgb_train,
                      valid_sets=[lgb_train, lgb_val],
                      num_boost_round=params['num_boost_round'],
                      #early_stopping_rounds=params['early_stopping_rounds'],
                      verbose_eval=False)
    y_pred = model.predict(test_sets[test].drop("date", axis=1), num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(test_sets[test]['usage'], y_pred))
    return rmse


In [427]:
rmse_scores_lgbm_tuned = list()
model_lgbm_params = dict()
for tr, te in zip(train_sets.keys(), test_sets.keys()):
    print(f"Process started for {tr[-1]}. Municipality")
    trials = Trials()
    best = fmin(
    fn=lambda params: lgbm_objective(params, tr, te),
    space = params,
    algo=tpe.suggest,
    max_evals=200,
    trials=trials,
    verbose=False)

    best_rmse = trials.results[np.argmin([r['loss'] for r in trials.results])]['loss']
    print(f"Best RMSE score for {tr[-1]}. Municipality: {best_rmse}\nBest parameters for {tr[-1]}: {best}")
    # Best RMSE
    rmse_scores_lgbm_tuned.append(best_rmse)
    model_lgbm_params[f"model{tr[-1]}"] = best
    print("".center(50, "~"))

Process started for 0. Municipality
Best RMSE score for 0. Municipality: 24.103694436806485
Best parameters for 0: {'feature_fraction': 0.8168065938636165, 'learning_rate': 0.017169813416704073, 'max_depth': 5, 'num_boost_round': 1328, 'num_leaves': 73}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Process started for 1. Municipality
Best RMSE score for 1. Municipality: 0.9082285599971849
Best parameters for 1: {'feature_fraction': 0.7750792166911057, 'learning_rate': 0.03060013785530977, 'max_depth': 2, 'num_boost_round': 1083, 'num_leaves': 10}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Process started for 2. Municipality
Best RMSE score for 2. Municipality: 3.9748725239911638
Best parameters for 2: {'feature_fraction': 0.9848687463849223, 'learning_rate': 0.007281903032223717, 'max_depth': 4, 'num_boost_round': 1490, 'num_leaves': 53}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Process started for 3. Municipality
Best RMSE score for 3. Municipality: 7.58888815242

In [428]:
plot_scores(rmse_scores_lgb, rmse_scores_lgbm_tuned)

<IPython.core.display.Javascript object>

Final Model

In [429]:
# check the last week and the first week of the test set errors.
def final_model(train, test, params):
    lgb_train = lgb.Dataset(train_sets[train].drop("date", axis=1), label=train_sets[train]["usage"])
    lgb_val = lgb.Dataset(test_sets[test].drop("date", axis=1), label=test_sets[test]["usage"])
    model = lgb.train(params, lgb_train,
                          valid_sets=[lgb_train, lgb_val],
                          num_boost_round=params['num_boost_round'],
                          #early_stopping_rounds=params['early_stopping_rounds'],
                          verbose_eval=False)
    all_test = test_sets[test].drop("date", axis=1)
    first_week = all_test[:int(len(all_test)/2)]
    last_week = all_test[int(len(all_test)/2):]

    y_pred = model.predict(all_test, num_iteration=model.best_iteration)
    y_pred_first = model.predict(first_week, num_iteration=model.best_iteration)
    y_pred_last = model.predict(last_week, num_iteration=model.best_iteration)

    rmse_first = np.sqrt(mean_squared_error(test_sets[test]['usage'][:int(len(test_sets[test])/2)], y_pred_first))
    rmse_last = np.sqrt(mean_squared_error(test_sets[test]['usage'][int(len(test_sets[test])/2):], y_pred_last))
    print(f"First Week RMSE for {tr[-1]}. Municipality: {rmse_first}")
    print(f"Last Week RMSE for {tr[-1]}. Municipality:: {rmse_last}")
    return y_pred


In [430]:
for tr, te in zip(train_sets.keys(), test_sets.keys()):
    params = model_lgbm_params[f"model{tr[-1]}"]
    params["verbosity"] = -999
    y_pred = final_model(tr, te, params)

    plot_tr = train_sets[tr]["usage"]
    plot_tr.index = range(0, len(plot_tr))

    plot_test = test_sets[te]["usage"]
    plot_test.index = range(len(plot_tr), len(plot_tr)+len(plot_test))

    y_pred = pd.DataFrame(y_pred)
    y_pred.index = range(len(plot_tr), len(plot_tr)+len(plot_test))
    plt.figure(figsize=(10, 4))
    plt.plot(plot_tr)
    plt.plot(plot_test)
    plt.plot(y_pred)
    plt.show();
    print("".center(50, "~"))

First Week RMSE for 0. Municipality: 2.083711076897833
Last Week RMSE for 0. Municipality:: 34.02402561652405


<IPython.core.display.Javascript object>

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
First Week RMSE for 1. Municipality: 1.0642111175945572
Last Week RMSE for 1. Municipality:: 0.7191751744723003


<IPython.core.display.Javascript object>

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
First Week RMSE for 2. Municipality: 3.615213952141816
Last Week RMSE for 2. Municipality:: 4.304584909628212


...

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
First Week RMSE for 8. Municipality: 3.6027482217361184
Last Week RMSE for 8. Municipality:: 2.559592238425409


<IPython.core.display.Javascript object>

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
First Week RMSE for 9. Municipality: 15.815672178645032
Last Week RMSE for 9. Municipality:: 14.122837160578612


<IPython.core.display.Javascript object>

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
